All data downloaded from https://www.skysports.com/

In [1]:
league_seasons = [f"{2000+year}-{year+1}" for year in range(17, 23)]

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initiating a new Google Chrome window
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# The function below extracts the games of any given season of La Liga and
# returns the output in the form of a dataframe
def extract_games(league_season):
    # The URL is shapes in the form of
    # https://www.skysports.com/la-liga-results/2011-12
    url = f"https://www.skysports.com/la-liga-results/{league_season}"
    
    # Retrieving the URL in the Chrome window
    driver.get(url)
    
    # When the page loads, the table's empty and is later filled by AJAX calls.
    # So, we wait until there is an element by the id of for instance
    # "widgetLite-5" available on the page before proceeding to
    # extract the table.
    # The availability of the said element shows that the table is fully loaded
    # and filled.
    element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, 'widgetLite-10'))
    )

    # These are elements initially displayed on the page
    visible_elements = BeautifulSoup(driver.page_source, 'lxml')

    # These are elements that are hidden inside a script for the "show more" button
    hidden_elements = BeautifulSoup(
        driver.find_element(By.CSS_SELECTOR, 'script[data-role="load-more-content"]').get_attribute('innerHTML'),
        'lxml')
    
    # Listing all the 'DIV' tags each containing the data for a single match
    all_match_divs = (visible_elements.findAll('div', attrs={"class": 'fixres__item'}) +
                   hidden_elements.find_all('div', attrs={"class": 'fixres__item'}))
    
    # Going through all the matches and extracting each one's info
    last_date = ''
    all_matches = []
    for match_div in all_match_divs:
        # The link to the match's specific webpage
        stats_link = match_div.find('a').get('href')
        
        # The names of the two teams
        team_names = [team.text for team in match_div.findAll('span', attrs={"class": 'swap-text__target'})]
        
        # If the previous sibling of the current 'DIV' element (single match) is an 'h4' element, get the
        # date of the match from it, otherwise the date of the match is the same as the previous match.
        if match_div.previous_sibling is None:
            match_date = last_date
        else:
            previous_element = match_div.previous_sibling.previous_sibling
            if previous_element.name == 'h4':
                match_date = previous_element.text 
                last_date = match_date
            else:
                match_date = last_date
        
        # Appending the data of the current match to the list of all matches
        all_matches.append([
            league_season,
            match_date,
            team_names[0],
            team_names[1],
            stats_link
        ])

    result = pd.DataFrame(all_matches, columns=[
        'season',
        'date',
        'home',
        'away',
        'link'])

    return result;

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:07<00:00, 996kB/s] 


In [3]:
df = pd.concat(
    [extract_games(league_season) for league_season in league_seasons])

df = df.reset_index(drop=True)

TimeoutException: Message: 
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0120DCE3+50899]
	(No symbol) [0x0119E111]
	(No symbol) [0x010A5588]
	(No symbol) [0x010D08F9]
	(No symbol) [0x010D0AFB]
	(No symbol) [0x010FF902]
	(No symbol) [0x010EB944]
	(No symbol) [0x010FE01C]
	(No symbol) [0x010EB6F6]
	(No symbol) [0x010C7708]
	(No symbol) [0x010C886D]
	GetHandleVerifier [0x01473EAE+2566302]
	GetHandleVerifier [0x014A92B1+2784417]
	GetHandleVerifier [0x014A327C+2759788]
	GetHandleVerifier [0x012A5740+672048]
	(No symbol) [0x011A8872]
	(No symbol) [0x011A41C8]
	(No symbol) [0x011A42AB]
	(No symbol) [0x011971B7]
	BaseThreadInitThunk [0x75F87D49+25]
	RtlInitializeExceptionChain [0x7708B74B+107]
	RtlClearBits [0x7708B6CF+191]


The stats' URL has a  subdirectory of 'stats' which should be inserted before id:
"https://www.skysports.com/football/espanyol-vs-sevilla/244855" should become:
"https://www.skysports.com/football/espanyol-vs-sevilla/stats/244855"

In [ ]:
import re

# This function returns the stats' URL of a given match's URL
def get_stats_url(link):
    id_pos = re.search(r"\d*$", link).start()
    return f"{link[:id_pos]}stats/{link[id_pos:]}"

# Going through each link and extracting the stats' link
df['stats_link'] = df['link'].apply(
    lambda link: get_stats_url(link)
)

Adding columns for match stats:

In [ ]:
df.to_csv("./initial_ds/match_stat_links.csv", index=False)